Question 1

In [203]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [204]:
auto = pd.read_csv('Auto.csv', na_values="?")
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [205]:
auto.shape

(397, 9)

In [206]:
auto_2 = auto.iloc[0:397, 0:(len(auto.columns)-1)]
median = auto_2['mpg'].median()
auto_2['mpg_high'] = auto_2['mpg'].apply(lambda x: 1 if x >= median else 0)
auto_2 = auto_2.dropna()
auto_2.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,0


In [207]:
Xvars = auto_2.ix[:, (1, 2, 3, 4, 5, 6, 7)].values
yvals = auto_2.ix[:, 8].values

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


(a)

In [208]:
from sklearn.model_selection import KFold
k = 4
kf_log = KFold(n_splits=4, shuffle=True, random_state=15)
kf_log.get_n_splits(X)

MSE_vec_kf = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    print('k index=', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    k_ind += 1
    print(classification_report(y_test, y_pred))


MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('The MSE of the model as the average MSE across the k = 4 test sets is', MSE_kf)
print('When k index = 0:')
print('Error rate of category mpg_high = 0 is 0.06, which is (1-0.94)')
print('Error rate of category mpg_high = 1 is 0.13, which is (1-0.87)')
print('When k index = 1:')
print('Error rate of category mpg_high = 0 is 0.12, which is (1-0.88)')
print('Error rate of category mpg_high = 1 is 0.08, which is (1-0.92)')
print('When k index = 2:')
print('Error rate of category mpg_high = 0 is 0.15, which is (1-0.85)')
print('Error rate of category mpg_high = 1 is 0.12, which is (1-0.88)')
print('When k index = 3:')
print('Error rate of category mpg_high = 0 is 0.05, which is (1-0.95)')
print('Error rate of category mpg_high = 1 is 0.15, which is (1-0.85)')

k index= 0
MSE for test set 0  is 0.09183673469387756
             precision    recall  f1-score   support

          0       0.94      0.89      0.92        55
          1       0.87      0.93      0.90        43

avg / total       0.91      0.91      0.91        98

k index= 1
MSE for test set 1  is 0.10204081632653061
             precision    recall  f1-score   support

          0       0.88      0.91      0.90        47
          1       0.92      0.88      0.90        51

avg / total       0.90      0.90      0.90        98

k index= 2
MSE for test set 2  is 0.1326530612244898
             precision    recall  f1-score   support

          0       0.85      0.87      0.86        45
          1       0.88      0.87      0.88        53

avg / total       0.87      0.87      0.87        98

k index= 3
MSE for test set 3  is 0.10204081632653061
             precision    recall  f1-score   support

          0       0.95      0.84      0.89        49
          1       0.85      0.96 

(b)

In [209]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
y = auto_2['mpg_high']
X = auto_2[['cylinders', 'displacement', 'horsepower', 'weight','acceleration', 'year', 'origin']]
auto_cl = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)
auto_cl.fit(Xvars, yvals)
y_pred = auto_cl.oob_decision_function_
y_pred_lst = []
for i in y_pred:
    if i[1] >= 0.5:
        y_pred_lst.append(1)
    else:
        y_pred_lst.append(0)
MSE = mean_squared_error(yvals, y_pred_lst)
print('The MSE of the random forest model as the MSE from the .oob prediction object = ', MSE)
print(classification_report(yvals, y_pred_lst))
print('Error rate of category mpg_high = 0', 0.06, ', which is (1 - 0.94)')
print('Error rate of category mpg_high = 1', 0.08, ', which is (1 - 0.92)')

The MSE of the random forest model as the MSE from the .oob prediction object =  0.07142857142857142
             precision    recall  f1-score   support

          0       0.94      0.91      0.93       196
          1       0.92      0.94      0.93       196

avg / total       0.93      0.93      0.93       392

Error rate of category mpg_high = 0 0.06 , which is (1 - 0.94)
Error rate of category mpg_high = 1 0.08 , which is (1 - 0.92)


(c)

In [210]:
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import svm, datasets
k = 4
kf_svm = KFold(n_splits=4, shuffle=True, random_state=15)

MSE_vec_svc = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    print('k index=', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    svc_rbf = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc_rbf.fit(X_train, y_train)
    y_pred = svc_rbf.predict(X_test)
    MSE_vec_svc[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    k_ind += 1
    print(classification_report(y_test, y_pred))


MSE_svc = MSE_vec_svc.mean()
MSE_kf_std = MSE_vec_svc.std()
print('The MSE of the model as the average MSE across the k = 4 test sets is', MSE_svc,', and the standard deviation \
of MSE is', MSE_kf_std)
print('When k index = 0:')
print('Error rate of category mpg_high = 0 is', 1-1)
print('Error rate of category mpg_high = 1 is', 1-0.45)
print('When k index = 1:')
print('Error rate of category mpg_high = 0 is', 1-0.48)
print('Error rate of category mpg_high = 1 is', 1-0)
print('When k index = 2:')
print('Error rate of category mpg_high = 0 is', 1-0.47)
print('Error rate of category mpg_high = 1 is', 1-1)
print('When k index = 3:')
print('Error rate of category mpg_high = 0 is', 1-0.53)
print('Error rate of category mpg_high = 1 is', 1-1)

k index= 0
MSE for test set 0  is 0.09183673469387756
             precision    recall  f1-score   support

          0       1.00      0.04      0.07        55
          1       0.45      1.00      0.62        43

avg / total       0.76      0.46      0.31        98

k index= 1
MSE for test set 1  is 0.10204081632653061
             precision    recall  f1-score   support

          0       0.48      1.00      0.65        47
          1       0.00      0.00      0.00        51

avg / total       0.23      0.48      0.31        98

k index= 2
MSE for test set 2  is 0.1326530612244898
             precision    recall  f1-score   support

          0       0.47      1.00      0.64        45
          1       1.00      0.04      0.07        53

avg / total       0.76      0.48      0.33        98

k index= 3
MSE for test set 3  is 0.10204081632653061
             precision    recall  f1-score   support

          0       0.53      1.00      0.69        49
          1       1.00      0.10 

/Users/apple/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(d)

I think Random Forest is the best predictor of mpg_high, because its MSE (0.07142857142857142) is much lower than MSEs of LogisticRegression (0.10714285714285715) and SVC (0.5076530612244898).